In [32]:
import spacy
from __future__ import print_function

nlp = spacy.load('en')

text = u'1 ounce of griled chiken'

In [35]:
doc = nlp(text)

In [36]:
for np in doc.noun_chunks:
    print(np.text)
    # print(np.text, np.root.text, np.root.dep_, np.root.head.text)


1 ounce
griled chiken


https://nicschrading.com/project/Intro-to-NLP-with-spaCy/
https://explosion.ai/blog/chatbot-node-js-spacy
https://github.com/explosion/spaCy/blob/master/examples/training/train_new_entity_type.py